In [20]:
# 更新 langchain 相关包以解决版本兼容问题
!pip install -U langchain-openai langchain-core langchain langchain-community langchain-experimental langchain-text-splitters langchain-classic

  Using cached langchain_openai-1.0.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached langchain_core-1.0.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached langchain-1.0.3-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached langsmith-0.4.38-py3-none-any.whl.metadata (14 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_experimental-0.3.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_experimental-0.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_experimental-0.3.1.post1-py3-none-any.whl.metadata (1.7 kB)
  Using c

In [ ]:
# def parse_args():
#     parse = argparse.ArgumentParser(description="RAG")
#     parse.add_argument("--embedding_model", type=str, default="BAAI/bge-m3", help="向量模型名称")
#     parse.add_argument("--embedding_model_dir", type=str, default="e:/pythonCode/Model/embedding", help="向量模型目录")
#     parse.add_argument("--model", type=str, default="gpt-5", help="大语言模型名称")
#     parse.add_argument("--base_url", type=str, default="https://api.chatanywhere.tech/v1", help="请求url")
#     parse.add_argument("--api_key", type=str, default="", help="API Key")
#     parse.add_argument("--embedding_vector_dir", type=str, default="faiss_db", help="向量数据库存储目录")
#     parse.add_argument("--model_provider", type=str, default="openai", help="模型提供商")
#     parse.add_argument("--chunk_size", type=int, default=1000, help="文本块大小")
#     parse.add_argument("--chunk_overlap", type=int, default=200, help="文本块重叠大小")
#     parse.add_argument("--if_download_model", type=bool, default=False, help="是否下载模型到本地")
#     args = parse.parse_args()
#     return args

## RAG
支持多个PDF上传，自动实现文本提取、分块、向量化

LangChain框架

阿里云百炼提供的 text-embedding-v1 向量嵌入 

基于FAISS构建检索数据库

DeepSeek-Chat模型进行回答

前端基于 Streamlit

阿里百炼的api_key以及DeepSeek的api key

pip install streamlit PyPDF2 dashscope faiss-cpu

**Free API : https://github.com/chatanywhere/GPT_API_free**

In [ ]:
# pip install streamlit PyPDF2 dashscope faiss-cpu
# pip install langchain_experimental


  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/10.1 MB 2.2 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/10.1 MB 2.2 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/10.1 MB 2.2 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/10.1 MB 2.2 MB/s eta 0:00:05
   ------- -------------------------------- 1.8/10.1 MB 1.7 MB/s eta 0:00:05
   --------------------------- ------------ 6.8/10.1 MB 5.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/10.1 MB 5.9 MB/s eta 0:00:01
   --------------------------------- ------ 8.4/10.1 MB 5.1 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.1 MB 5.0 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.1 MB 5.0 MB/s eta 0:00:01
   ----------------------

In [21]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.tools import create_retriever_tool
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.chat_models import init_chat_model

# 这里由于LangChain的一些更新，使用 langchain_classic 来获取传统的 Agent API
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor


✅ 所有依赖导入成功!


In [ ]:
llm = init_chat_model(
    model='gpt-5', # 模型名称
    model_provider='openai', # 模型提供商
    base_url="https://api.chatanywhere.tech/v1", #请求url
    api_key="", 
)

In [23]:
from modelscope import AutoModel, snapshot_download

In [24]:
model_id="BAAI/bge-m3"
cache_dir="e:/pythonCode/Model/embedding"

In [25]:
embeddings = AutoModel.from_pretrained(cache_dir+'/'+model_id)

In [ ]:
from langchain.chat_models import init_chat_model
model = init_chat_model(
    model="gpt-5", # 模型名称
    model_provider="openai", # 模型提供商，硅基流动提供了openai请求格式的访问
    base_url="https://api.chatanywhere.tech/v1", #硅基流动模型的请求url
    api_key="", # 填写你注册的硅基流动 API Key
)


In [ ]:
#读取pdf上传的内容
def pdf_read(pdf_doc):
    text = ""
    for pdf in pdf_doc:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
# 将向量存储在本地FAISS数据库中
def vector_store(text_chunks):
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_db")

In [ ]:
import os
# 构建知识库回答逻辑链
def check_database_exists():
    """检查FAISS数据库是否存在"""
    return os.path.exists("faiss_db") and os.path.exists("faiss_db/index.faiss")

def user_input(user_question):
    # 检查数据库是否存在
    if not check_database_exists():
        st.error("❌ 请先上传PDF文件并点击'Submit & Process'按钮来处理文档！")
        st.info("  步骤：1️⃣ 上传PDF → 2️⃣ 点击处理 → 3️⃣ 开始提问")
        return

    try:
        # 加载FAISS数据库
        new_db = FAISS.load_local("faiss_db", embeddings, allow_dangerous_deserialization=True)
        # 将数据转化为LangChain检索工具
        retriever = new_db.as_retriever() 
        
        # 使用LangChain的create_retriever_tool构建内容检索工具
        retrieval_chain = create_retriever_tool(retriever, "pdf_extractor",
                                                "This tool is to give answer to queries from the pdf")
        get_conversational_chain(retrieval_chain, user_question)

    except Exception as e:
        st.error(f"❌ 加载数据库时出错: {str(e)}") # 前端界面报错
        st.info("请重新处理PDF文件") # 前端界面info提示

def get_conversational_chain(tools, querys):
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """你是AI助手，请根据提供的上下文回答问题，确保提供所有细节，如果答案不在上下文中，请说"答案不在上下文中"，不要提供错误的答案""",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])

    tool = [tools]
    agent = create_tool_calling_agent(llm, tool, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)

    response = agent_executor.invoke({"input": querys})
    print(response)
    st.write("  回答: ", response['output'])

## UI界面

运行 streamlit run pythonfile 来启动

In [ ]:
def main():
    st.set_page_config("  LangChain知识库系统开发")
    st.header("  LangChain知识库系统开发")

    # 显示数据库状态
    col1, col2 = st.columns([3, 1])

    with col1:
        if check_database_exists():
            pass
        else:
            st.warning("⚠️ 请先上传并处理PDF文件")

    with col2:
        if st.button(" ️ 清除数据库"):
            try:
                import shutil
                if os.path.exists("faiss_db"):
                    shutil.rmtree("faiss_db")
                st.success("数据库已清除")
                st.rerun()
            except Exception as e:
                st.error(f"清除失败: {e}")

    # 用户问题输入
    user_question = st.text_input("  请输入问题",
                                  placeholder="例如：这个文档的主要内容是什么？",
                                  disabled=not check_database_exists())

    if user_question:
        if check_database_exists():
            with st.spinner("  AI正在分析文档..."):
                user_input(user_question)  # user_input里包含了将user_question输入到大模型里的代码
        else:
            st.error("❌ 请先上传并处理PDF文件！")

    # 侧边栏
    with st.sidebar:
        st.title("  文档管理")

        # 显示当前状态
        if check_database_exists():
            st.success("✅ 数据库状态：已就绪")
        else:
            st.info("  状态：等待上传PDF")

        st.markdown("---")

        # 文件上传
        pdf_doc = st.file_uploader(
            "  上传PDF文件",
            accept_multiple_files=True,
            type=['pdf'],
            help="支持上传多个PDF文件"
        )

        if pdf_doc:
            st.info(f"  已选择 {len(pdf_doc)} 个文件")
            for i, pdf in enumerate(pdf_doc, 1):
                st.write(f"{i}. {pdf.name}")

        # 处理按钮
        process_button = st.button(
            "  提交并处理",
            disabled=not pdf_doc,
            use_container_width=True
        )

        if process_button:
            if pdf_doc:
                with st.spinner("  正在处理PDF文件..."):
                    try:
                        # 读取PDF内容
                        raw_text = pdf_read(pdf_doc)

                        if not raw_text.strip():
                            st.error("❌ 无法从PDF中提取文本，请检查文件是否有效")
                            return

                        # 分割文本
                        text_chunks = get_chunks(raw_text)
                        st.info(f"  文本已分割为 {len(text_chunks)} 个片段")

                        # 创建向量数据库
                        vector_store(text_chunks)

                        st.success("✅ PDF处理完成！现在可以开始提问了")
                        st.balloons()
                        st.rerun()

                    except Exception as e:
                        st.error(f"❌ 处理PDF时出错: {str(e)}")
            else:
                st.warning("⚠️ 请先选择PDF文件")

        # 使用说明
        with st.expander("  使用说明"):
            st.markdown("""
                **步骤：**
                1.   上传一个或多个PDF文件
                2.   点击"Submit & Process"处理文档
                3.   在主页面输入您的问题
                4.   AI将基于PDF内容回答问题
    
                **提示：**
                - 支持多个PDF文件同时上传
                - 处理大文件可能需要一些时间
                - 可以随时清除数据库重新开始
                """)
if __name__ == "__main__":
    main()